In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample  # For Doppler shift simulation

# -----------------------------------------------------------------------------
# Step 1: Load Predefined UWA Channel Parameters 
# -----------------------------------------------------------------------------
def get_uwa_channel_params(channel_type="shallow_coastal"):
    """
    Get UWA channel parameters (multipath, Doppler, noise) 
    Args:
        channel_type: "shallow_coastal" (0–50 m) or "continental_shelf" (50–200 m)
    Returns:
        channel_params: Dictionary of channel parameters (per paper)
    """
    if channel_type == "shallow_coastal":
        # Shallow Coastal Channel (Table 3: Inputs/Outputs)
        return {
            "num_multipath": 28,          # Number of multipath components
            "delay_spread_ms": 132.9,     # Delay spread (ms)
            "doppler_shift_hz": 0.6,     # Doppler shift (Hz)
            "tx_loss_db": 60,             # Transmission loss (dB)
            "noise_type": "ship",         # Ship-induced AWGN (Table 3)
            "noise_snr_range_db": (10, 20)# Noise SNR (1–5 kHz band; Table 3)
        }
    elif channel_type == "continental_shelf":
        # Continental Shelf Channel (Table 3: Inputs/Outputs)
        return {
            "num_multipath": 63,          # Number of multipath components
            "delay_spread_ms": 100.0,     # Delay spread (ms)
            "doppler_shift_hz": 0.45,    # Doppler shift (Hz)
            "tx_loss_db": 80,             # Transmission loss (dB)
            "noise_type": "mixed",        # Mixed ship/marine noise (Table 3)
            "noise_snr_range_db": (5, 15) # Noise SNR (10–100 Hz band; Table 3)
        }
    else:
        raise ValueError("Channel type must be 'shallow_coastal' or 'continental_shelf'")


# -----------------------------------------------------------------------------
# Step 2: Generate UWA Channel Impulse Response (CIR) (per Bellhop Model)
# -----------------------------------------------------------------------------
def generate_uwa_cir(channel_params, fs):
    """
    Generate UWA channel impulse response (CIR) using Bellhop-inspired multipath.
    Args:
        channel_params: UWA channel parameters (from get_uwa_channel_params)
        fs: Sampling frequency (Hz) of the transmitter signal
    Returns:
        h: Complex CIR (time-domain; includes multipath gains/delays)
        delays_samples: Multipath delays (in samples)
    """
    num_multipath = channel_params["num_multipath"]
    delay_spread_ms = channel_params["delay_spread_ms"]
    
    # 1. Generate random multipath delays (uniformly distributed over delay spread)
    max_delay_samples = int(delay_spread_ms * 1e-3 * fs)  # Convert ms to samples
    delays_samples = np.random.uniform(0, max_delay_samples, num_multipath)
    delays_samples = np.sort(delays_samples)  # Sort delays (earlier to later paths)
    
    # 2. Generate multipath gains (complex; decay with delay, per UWA propagation)
    # Gain decays exponentially with delay (matches Bellhop's transmission loss model)
    delay_decay = np.exp(-delays_samples / max_delay_samples)  # Decay factor
    gain_magnitude = np.random.normal(0.5, 0.1, num_multipath) * delay_decay
    gain_phase = np.random.uniform(-np.pi, np.pi, num_multipath)  # Random phase
    gains = gain_magnitude * np.exp(1j * gain_phase)
    
    # 3. Create CIR (time-domain: impulse at each delay with corresponding gain)
    cir_length = max_delay_samples + 1  # CIR length = max delay + 1 sample
    h = np.zeros(cir_length, dtype=np.complex128)  # NumPy 2.0 compatible
    for gain, delay in zip(gains, delays_samples):
        h[int(round(delay))] += gain  # Assign gain to delay index
    
    # 4. Apply transmission loss (dB to linear scale)
    tx_loss_linear = 10 ** (-channel_params["tx_loss_db"] / 20)
    h *= tx_loss_linear
    
    return h, delays_samples


# -----------------------------------------------------------------------------
# Step 3: Apply Doppler Shift (per UWA Channel Dynamics)
# -----------------------------------------------------------------------------
def apply_doppler_shift(signal, fs, doppler_shift_hz):
    """
    Apply Doppler shift to UWA signal (frontiers.pdf Section 2: Doppler spread).
    Args:
        signal: Input time-domain signal (complex or real)
        fs: Sampling frequency (Hz)
        doppler_shift_hz: Doppler shift (Hz; positive = frequency increase)
    Returns:
        signal_doppler: Signal with Doppler shift applied
    """
    # Doppler shift = frequency offset → resample signal (scaled time)
    # Time scaling factor: 1 + (doppler_shift_hz / fs) (small Doppler approximation)
    time_scale = 1 + (doppler_shift_hz / fs)
    num_samples_new = int(round(len(signal) * time_scale))
    
    # Resample to apply time scaling (equivalent to frequency shift for narrowband signals)
    signal_doppler = resample(signal, num_samples_new)
    
    # Trim/pad to match original length (for alignment with transmitter signal)
    if len(signal_doppler) > len(signal):
        signal_doppler = signal_doppler[:len(signal)]
    else:
        signal_doppler = np.pad(signal_doppler, (0, len(signal) - len(signal_doppler)), mode="constant")
    
    return signal_doppler


# -----------------------------------------------------------------------------
# Step 4: Apply AWGN Noise (per UWA Channel Noise Model)
# -----------------------------------------------------------------------------
def add_uwa_awgn(signal, fs, channel_params, target_snr_db):
    """
    Add AWGN noise to UWA signal (matches paper's noise models: ship/mixed; ).
    Args:
        signal: Input signal (complex or real)
        fs: Sampling frequency (Hz)
        channel_params: UWA channel parameters (from get_uwa_channel_params)
        target_snr_db: Desired SNR (dB) (within paper's range: 0–25 dB; )
    Returns:
        signal_noisy: Signal with AWGN added
    """
    # 1. Calculate signal power
    if np.iscomplexobj(signal):
        signal_power = np.mean(np.abs(signal) ** 2)
    else:
        signal_power = np.mean(signal ** 2)
    
    # 2. Calculate noise power (from target SNR)
    noise_power = signal_power / (10 ** (target_snr_db / 10))
    
    # 3. Generate AWGN (complex if signal is complex, real otherwise)
    if np.iscomplexobj(signal):
        noise = np.random.normal(0, np.sqrt(noise_power / 2), len(signal)) + \
                1j * np.random.normal(0, np.sqrt(noise_power / 2), len(signal))
    else:
        noise = np.random.normal(0, np.sqrt(noise_power), len(signal))
    
    # 4. Match noise to paper's frequency band (e.g., 1–5 kHz for shallow coastal)
    # Apply bandpass filter to noise (matches paper's noise spectral characteristics)
    from scipy.signal import firwin, lfilter
    if channel_params["noise_type"] == "ship":
        # Ship noise: 1–5 kHz band (Table 3)
        cutoff = [1000, 5000]
    else:  # mixed
        # Mixed noise: 10–100 Hz band (Table 3)
        cutoff = [10, 100]
    
    # FIR bandpass filter (order 100; Nyquist frequency = fs/2)
    nyq = 0.5 * fs
    cutoff_norm = [f / nyq for f in cutoff]
    filter_coeff = firwin(100, cutoff_norm, pass_zero=False)
    noise_filtered = lfilter(filter_coeff, 1.0, noise)
    
    # 5. Add filtered noise to signal
    signal_noisy = signal + noise_filtered
    return signal_noisy


# -----------------------------------------------------------------------------
# Step 5: Full UWA Channel Propagation Pipeline 
# -----------------------------------------------------------------------------
def uwa_channel_propagation(tx_signal, fs, channel_type="shallow_coastal", target_snr_db=10):
    """
    Simulate full UWA channel propagation (transmitter signal → receiver signal) per frontiers.pdf.
    Args:
        tx_signal: Transmitter signal (real-valued baseband; from OFDM transmitter)
        fs: Sampling frequency (Hz) of tx_signal
        channel_type: "shallow_coastal" or "continental_shelf" (per paper)
        target_snr_db: Target SNR (0–25 dB; Table 4: SNR Range = 0:5:25 dB)
    Returns:
        rx_signal: Received signal (after multipath, Doppler, noise)
        cir: UWA channel impulse response (for channel estimation, per paper)
    """
    # 1. Get UWA channel parameters 
    channel_params = get_uwa_channel_params(channel_type)
    
    # 2. Convert real transmitter signal to complex (for multipath/Doppler processing)
    tx_signal_complex = tx_signal.astype(np.complex128)  # NumPy 2.0 compatible
    
    # 3. Generate UWA channel impulse response (CIR) (Bellhop-inspired; Section 3.1)
    cir, _ = generate_uwa_cir(channel_params, fs)
    
    # 4. Apply multipath: Convolve signal with CIR (time-domain)
    # Use circular convolution to avoid edge effects (per UWA-OFDM design)
    signal_multipath = np.convolve(tx_signal_complex, cir, mode="same")
    
    # 5. Apply Doppler shift (UWA channel is doubly selective; Section 2)
    signal_doppler = apply_doppler_shift(signal_multipath, fs, channel_params["doppler_shift_hz"])
    
    # 6. Apply AWGN noise (matches paper's noise model; Table 3)
    rx_signal_complex = add_uwa_awgn(signal_doppler, fs, channel_params, target_snr_db)
    
    # 7. Convert back to real-valued (UWA receiver processes real signals; per paper)
    rx_signal = np.real(rx_signal_complex)
    
    return rx_signal, cir